<a href="https://colab.research.google.com/github/smatiolids/astra-vector-notebooks/blob/main/Astra_DB_Vector_Security_for_data_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Astra DB Vector - How to set security per Collection

Astra is a secure DB for sensitive data. Its role based structure allows to define granular access control.

In this example, we will see how to define refined access permission for different apps and users.

In [ ]:
!pip install astrapy --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Init variables.
import os, json
from getpass import getpass

ASTRA_ORG_ID=input("ASTRA_ORG_ID: ")
ASTRA_DB_ID=input("ASTRA_DB_ID: ")
ASTRA_DB_API_ENDPOINT=input("ASTRA_DB_ENDPOINT: ")
ASTRA_KEYSPACE=input("ASTRA_KEYSPACE: ")
ASTRA_COLLECTION_ALLOWED=input("Allowed Collection: ")
ASTRA_COLLECTION_DENIED=input("Denied Collection: ")

ASTRA_ORG_ID: 41940237-476e-46b6-9971-1615fcb28048
ASTRA_DB_ID: b0748576-a92d-4682-86b0-13a0a04fb4dd
ASTRA_DB_ENDPOINT: https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com
ASTRA_KEYSPACE: default_keyspace
Allowed Collection: ecommerce_products
Denied Collection: review_analytics


In [ ]:
# Tokens generated from inside the database UI doesn't have access to roles and to create new tokens
ASTRA_ORG_ADMIN_TOKEN = getpass("ASTRA_ORG_ADMIN_TOKEN (Generated with Organization Administrator Role) = ")

ASTRA_ORG_ADMIN_TOKEN (Generated with Organization Administrator Role) = ··········


In [ ]:
# Function to send requests to Astra DevOps API
import requests
import pandas as pd

def astra_devops(url, method = 'GET', data = {}):
    headers = {'Accept': 'application/json', 'Authorization': f'Bearer {ASTRA_ORG_ADMIN_TOKEN}'}
    res = requests.request(method, url=url, headers=headers, data=json.dumps(data))

    if int(res.status_code) >= 400:
      return res.text

    try:
        res_data = res.json()
    except ValueError:
        res_data = res.status_code

    if isinstance(res_data, list):
        return pd.DataFrame.from_records(res_data)

    return res_data

In [ ]:
# Checking existent roles
df_roles = astra_devops('https://api.astra.datastax.com/v2/organizations/roles')
df_roles.head()


,id,name,policy,last_update_date_time,last_update_user_id
0,b4ed0e9e-67e8-47b6-8b58-c6629be961a9,R/W Svc Acct,"{'description': 'R/W Svc Acct', 'resources': [...",0001-01-01T00:00:00Z,
1,43745b73-ad46-46e4-b826-c15d06d2cea0,Admin User,"{'description': 'Admin User', 'resources': ['d...",0001-01-01T00:00:00Z,
2,67c4b5dc-dd3f-4b2d-be51-09be12836d57,API Admin User,"{'description': 'API Admin User', 'resources':...",0001-01-01T00:00:00Z,
3,ad0566b5-2a67-49de-89e8-92258c2f2c98,Organization Administrator,"{'description': 'Organization Administrator', ...",0001-01-01T00:00:00Z,
4,16a4b1d7-a615-41f8-95ca-52b0280f4d77,RO Svc Acct,"{'description': 'RO Svc Acct', 'resources': ['...",0001-01-01T00:00:00Z,


## Role definition

The access to the database is done based on roles and tokens.

As our goal is to have an app that can access only one collection of documents, we need to define a role to determine the permissions for the specific resources.

The role needs to have access to the organization, the database, some metadata views and the table itself. This is defined in this policy.


In [ ]:
#creating a Role for the collection 1
def get_role_definition(table, org = ASTRA_ORG_ID, db = ASTRA_DB_ID, ks = ASTRA_KEYSPACE, name='Role' ):
    role_definition = {
            "name": f"read_{table}_20",
            "policy": {
            "description": "Access to view data from table/collection",
            "resources": [
                f"drn:astra:org:{org}",
                f"drn:astra:org:{org}:db:{db}",
                f"drn:astra:org:{org}:db:{db}:keyspace:system_virtual_schema",
                f"drn:astra:org:{org}:db:{db}:keyspace:system_virtual_schema:table:keyspaces",
                f"drn:astra:org:{org}:db:{db}:keyspace:system_virtual_schema:table:tables",
                f"drn:astra:org:{org}:db:{db}:keyspace:system_virtual_schema:table:columns",

                ## Tables allowed for read
                f"drn:astra:org:{org}:db:{db}:keyspace:{ks}",
                f"drn:astra:org:{org}:db:{db}:keyspace:{ks}:table:{table}"
            ],
            "actions": [
                "org-db-view",
                "db-all-keyspace-describe",
                "db-keyspace-describe",
                "db-keyspace-authorize",
                "db-table-select",
                "db-table-describe",
                "db-table-authorize",
                "db-rest",
                "db-cql"
            ],
            "effect": "allow"}
        }
    return role_definition


In [ ]:
# Before creating, let's inspect the role definition
role = get_role_definition(ASTRA_COLLECTION_ALLOWED)
role

{'name': 'read_ecommerce_products_20',
 'policy': {'description': 'Access to view data from table/collection',
  'resources': ['drn:astra:org:41940237-476e-46b6-9971-1615fcb28048',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema:table:keyspaces',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema:table:tables',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema:table:columns',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:default_keyspace',
   'drn:astra:org:41940237-476e-46b6-9971-16

In [ ]:
# Creating the role
res_role = astra_devops('https://api.astra.datastax.com/v2/organizations/roles', method='POST', data=get_role_definition(ASTRA_COLLECTION_ALLOWED))
res_role
role_id = res_role['id']
print(f"Role ID: {role_id}")


Role ID: 37ee4615-c990-4ab4-b846-dfe18b20fe12


In [ ]:
# Updating the role
res_role = astra_devops(f'https://api.astra.datastax.com/v2/organizations/roles/{role_id}',
                      method='PUT',
                      data=get_role_definition(ASTRA_COLLECTION_ALLOWED))
print(f"Update response: {res_role}")

Update response: 200


In [ ]:
# Get role definition
# Checking the current role version
res_role = astra_devops(f'https://api.astra.datastax.com/v2/organizations/roles/{role_id}',
                        method='GET')
res_role

{'id': '37ee4615-c990-4ab4-b846-dfe18b20fe12',
 'name': 'read_ecommerce_products_20',
 'policy': {'description': 'Access to view data from table/collection',
  'resources': ['drn:astra:org:41940237-476e-46b6-9971-1615fcb28048',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema:table:keyspaces',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema:table:tables',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:system_virtual_schema:table:columns',
   'drn:astra:org:41940237-476e-46b6-9971-1615fcb28048:db:b0748576-a92d-4682-86b0-13a0a04fb4dd:keyspace:default_keyspace

In [ ]:
# Generating a new token for the created role
import time

# If there is a token created, delete it before creating a new one
if 'res_token' in vars():
  remove_token = astra_devops(f"https://api.astra.datastax.com/v2/clientIdSecrets/{res_token['clientId']}",
                     method='DELETE')
  print(f"Token revoked: {res_token['clientId']} = {remove_token}")

#Generate a new token
res_token = astra_devops('https://api.astra.datastax.com/v2/clientIdSecrets',
                         method='POST',
                         data={"roles": [res_role['id']]}
)
print(f"New token: {res_token['clientId']}")

New token: lpIjzUGhAcmAmbjlWSbEndTW


## AstraPY - Getting Data

In [ ]:
from astrapy.db import AstraDB, AstraDBCollection

In [ ]:
# Getting data from the specified collection should work
from time import sleep

astra_db = AstraDB(
api_endpoint=ASTRA_DB_API_ENDPOINT,
token=res_token['token'],
)

collection = AstraDBCollection(
    collection_name=ASTRA_COLLECTION_ALLOWED, astra_db=astra_db
)

for i in range(20):
  try:
    res = collection.find_one()
    print(f"[{i}] Success:  {res['data']['document']['_id']}")
  except Exception as e:
    print(f"[{i}] Error: {e}")
  sleep(1)

[0] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[1] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[2] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[3] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[4] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[5] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[6] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[7] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[8] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[9] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[10] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[11] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[12] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[13] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[14] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[15] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[16] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[17] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[18] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9
[19] Success:  397c966e9c7503ca04c8cfe5a1a5f9c9


In [ ]:
# Getting data from the specified collection should fail
from time import sleep

astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=res_token['token'],
)
collection = AstraDBCollection(
    collection_name=ASTRA_COLLECTION_DENIED, astra_db=astra_db
)

for i in range(5):
  try:
    res = collection.find_one()
    print(f"[{i}] Success:  {res['data']['document']['_id']}")
  except Exception as e:
    # Failed, as expected
    print(f"[{i}] Error: {e}")
  sleep(1)

[0] Error: Client error '401 Unauthorized' for url 'https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com/api/json/v1/default_keyspace/review_analytics'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401
[1] Error: Client error '401 Unauthorized' for url 'https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com/api/json/v1/default_keyspace/review_analytics'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401
[2] Error: Client error '401 Unauthorized' for url 'https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com/api/json/v1/default_keyspace/review_analytics'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401
[3] Error: Client error '401 Unauthorized' for url 'https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com/api/json/v1/default_keyspace/review_analytics'
For more information chec